#### MP based asynchronous testing

In [1]:
import multiprocessing as mp
import TestPool_Unit
from shutil import copyfile
import numpy as np

In [2]:
def Evaluate(result_arr):
    print('Files Processed: ', len(result_arr))
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    for entry in result_arr:
        (word_match, lemma_match, n_dcsWords, n_output_nodes) = entry
        recalls.append(lemma_match/n_dcsWords)
        recalls_of_word.append(word_match/n_dcsWords)

        precisions.append(lemma_match/n_output_nodes)
        precisions_of_words.append(word_match/n_output_nodes)
    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))

In [11]:
modelFile = 'outputs/train_nnet_t882478786975.p'
# Backup the model file
copyfile(modelFile, modelFile + '.bk')

# Create Queue, Result array
queue = mp.Queue()
result_arr = []

# Start 6 workers - 8 slows down the pc
proc_count = 6
procs = [None]*proc_count
for i in range(proc_count):
    vpid = i
    procs[i] = mp.Process(target = TestPool_Unit.pooled_Test, args = (modelFile, vpid, queue, '../NewData/skt_dcs_DS.bz2_10K/', 700))
# Start Processes
for i in range(proc_count):
    procs[i].start()

In [53]:
# Fetch partial results
while not queue.empty():
    result_arr.append(queue.get())
# Evaluate results till now
Evaluate(result_arr)

Files Processed:  2462
Avg. Micro Recall of Lemmas: 0.8308170232199714
Avg. Micro Recall of Words: 0.695273642255833
Avg. Micro Precision of Lemmas: 0.7718980601724442
Avg. Micro Precision of Words: 0.6470046899265305


In [9]:
# Check status
for i in range(proc_count):
    p = procs[i]
    print('Process with\t vpid: {}\t ->\t pid: {}\t ->\t running status: {}'.format(i, p.pid, p.is_alive()))

Process with	 vpid: 0	 ->	 pid: 4836	 ->	 running status: False
Process with	 vpid: 1	 ->	 pid: 500	 ->	 running status: False
Process with	 vpid: 2	 ->	 pid: 8972	 ->	 running status: False
Process with	 vpid: 3	 ->	 pid: 4596	 ->	 running status: False
Process with	 vpid: 4	 ->	 pid: 272	 ->	 running status: False
Process with	 vpid: 5	 ->	 pid: 9432	 ->	 running status: False


In [8]:
# Properly Join
for i in range(proc_count):
    procs[i].join()

In [7]:
# Force Terminate
for p in procs:
    p.terminate()

In [865]:
np.mean([a[2] for a in result_arr])

8.3328244274809169